In [1]:
import time
import random
import cProfile

import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import folium

from routers import time_dependent_dijkstra, single_source_time_dependent_dijkstra
from converters import *
from functions import *
from loaders import create_GTFS_graph
from other import plot_path_browser

Задаем константы


In [2]:
GTFSpath = r"D:\Python_progs\Feeds\Klapeida" # Путь к папке с GTFS файлами
departure_time_input = "18:00:00" # Время отправления в формате HH:MM:SS
day = 'monday' # День недели
time_zero = time.time()

plot_graph = False # Не работает
save_city_graph = False # Сохранять граф города в файл
plot_path_in_browser = False # Отобразить маршрут при помощи Folium в браузере
plot_path_folium = True # Отобразить маршрут при помощи Folium в Ipython Notebook

Загружаем комбинированный граф GTFS и OSM

In [3]:
G_combined, city_name, departure_time_seconds, stops = create_GTFS_graph(GTFSpath, departure_time_input, 
                                                                         day, 
                                                                         duration_seconds=3600*3, 
                                                                         save_to_csv = True)

GTFS data loaded
12181 of 72974 trips retained
Выпуклая оболочка построена
Граф улиц загружен
Число узлов: 32002
Число ребер: 83167
Соединение завершено


Выбираем рандомные узлы графа для расчета времени в пути. 
По координатам потом удобно сверять время с Гугл картами/Яндексом  

p.s. Главное отключить метро :) 
p.p.s В гугл картах лучше сравнивать не время в пути, а время прибытия, т.к. они не показывают время ожидания



In [4]:
source = random.choice(list(G_combined.nodes))
target = random.choice(list(G_combined.nodes))

print("Source: ", source, "coordinates", G_combined.nodes[source]['y'], G_combined.nodes[source]['x'])
print("Target: ", target, "coordinates", G_combined.nodes[target]['y'], G_combined.nodes[target]['x'])

Source:  4206870440 coordinates 55.9369041 21.0865354
Target:  1147032473 coordinates 55.7013613 21.1509527


In [5]:
def main():
    sample_time = time.time()
    
    path, arrival_time, travel_time, used_routes = time_dependent_dijkstra(
                            graph = G_combined, 
                            source = source, 
                            target = target, 
                            start_time = parse_time_to_seconds(departure_time_input),
                            track_used_routes=True,
                        )

    #print(f"Path: {path}")
    print('Used routes:', used_routes)
    print(f"Arrival time at destination: {parse_seconds_to_time(arrival_time)} in {parse_seconds_to_time(travel_time)}")

    print(f"Time elapsed: {time.time() - sample_time} seconds")

    return path, arrival_time, travel_time

path, arrival_time, travel_time = main()

Used routes: {None, 'klaipeda_bus_3', 'klaipeda_bus_14', 'klaipeda_bus_31', 'klaipeda_bus_17'}
Arrival time at destination: 20:06:42 in 02:06:42
Time elapsed: 0.06900310516357422 seconds


In [6]:
G_path = nx.subgraph(G_combined, path)

graph = nx.MultiDiGraph(G_path)

G_nodes, G_edges = ox.graph_to_gdfs(graph)

frame_center_lat = stops['stop_lat'].mean()
frame_center_lon = stops['stop_lon'].mean()

if plot_path_in_browser:
    plot_path_browser(graph, stops)


m2 = folium.Map(location=[frame_center_lat, frame_center_lon], 
                zoom_start=12,
                width='100%', height='100%')

G_edges.explore(m = m2)
G_nodes.explore(m = m2,
                column = 'type')

In [7]:
calculate_OD_matrix(G_combined, stops, parse_time_to_seconds(departure_time_input))

In [ ]:
cProfile.run("main()") 

times_list = []
startt = parse_time_to_seconds(departure_time_input)
endt = startt + 3600
for i in range(startt, endt, 360):
    try:
        path, arrival_time, travel_time, used_routes = time_dependent_dijkstra_2(
                                        graph = G_combined, 
                                        source = source, 
                                        target = target, 
                                        start_time = i,
                                        track_used_routes=True)
                                    
        times_list.append(tuple([i, travel_time]))
    except:
        continue
    
print(times_list)

plt.figure(figsize=(14,7))
plt.plot(*zip(*times_list))

plt.xlabel('Departure Time')
plt.ylabel('Travel Time')
plt.show()